In [1]:
!pip install ultralytics

In [2]:
import os
import shutil
import cv2
import pandas as pd
import numpy as np
from IPython.display import HTML
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display
from ultralytics import YOLO
import torch

In [12]:
import kagglehub

path = kagglehub.dataset_download("alkanerturan/vehicledetection")

print("Path to dataset files:", path)

!ls {path}
os.environ['WANDB_DISABLED'] = 'true'

Path to dataset files: /root/.cache/kagglehub/datasets/alkanerturan/vehicledetection/versions/3
TestVideo  VehiclesDetectionDataset


In [13]:
train = "/root/.cache/kagglehub/datasets/alkanerturan/vehicledetection/versions/3/VehiclesDetectionDataset/train"
validation = "/root/.cache/kagglehub/datasets/alkanerturan/vehicledetection/versions/3/VehiclesDetectionDataset/valid"
test = "/root/.cache/kagglehub/datasets/alkanerturan/vehicledetection/versions/3/VehiclesDetectionDataset/test"
video = "/root/.cache/kagglehub/datasets/alkanerturan/vehicledetection/versions/3/TestVideo/TrafficPolice.mp4"
yaml = "/root/.cache/kagglehub/datasets/alkanerturan/vehicledetection/versions/3/VehiclesDetectionDataset/dataset.yaml"

In [5]:
def load_and_prepare_image(file_path, dimensions=(640, 640)):
    img = cv2.imread(file_path)
    img_resized = cv2.resize(img, dimensions)
    img_normalized = img_resized.astype('float32') / 255.0

    return img_normalized

def apply_augmentation(image):
    return cv2.flip(image, flipCode=1)

In [6]:
def create_dataset(directory, max_images=None):  # Adicionado parâmetro max_images
    images_dir = os.path.join(directory, 'images')
    labels_dir = os.path.join(directory, 'labels')

    dataset = []

    for filename in os.listdir(images_dir):
        img_filepath = os.path.join(images_dir, filename)
        label_filepath = os.path.join(labels_dir, filename.replace('.jpg', '.txt'))

        original_image = load_and_prepare_image(img_filepath)
        transformed_image = apply_augmentation(original_image)

        with open(label_filepath, 'r') as label_file:
            label_content = label_file.read().strip()

        dataset.append((original_image, label_content))
        dataset.append((transformed_image, label_content))

        # Adiciona limite no número de imagens
        if max_images and len(dataset) >= max_images:
            break

    return dataset

# Defina os tamanhos de amostragem
train_sample_size = 650  # Exemplo de tamanho reduzido
validation_sample_size = 250  # Exemplo de tamanho reduzido

# Criar conjuntos de dados amostrados
training_data = create_dataset(train, max_images=train_sample_size)
validation_data = create_dataset(validation, max_images=validation_sample_size)

In [7]:
torch.cuda.empty_cache()

In [8]:
model = YOLO('yolov8n.pt')
model.train(data=yaml, epochs=30, imgsz=640, batch=8, name='yolov8-vehicledetection')

Ultralytics 8.3.23 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/root/.cache/kagglehub/datasets/alkanerturan/vehicledetection/versions/3/VehiclesDetectionDataset/dataset.yaml, epochs=30, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8-vehicledetection3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames

train: Scanning /root/.cache/kagglehub/datasets/alkanerturan/vehicledetection/versions/3/VehiclesDetectionDataset/train/labels.cache... 878 images, 0 backgrounds, 0 corrupt: 100%|██████████| 878/878 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.20 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning /root/.cache/kagglehub/datasets/alkanerturan/vehicledetection/versions/3/VehiclesDetectionDataset/valid/labels.cache... 250 images, 0 backgrounds, 0 corrupt: 100%|██████████| 250/250 [00:00<?, ?it/s]


Plotting labels to runs/detect/yolov8-vehicledetection3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/yolov8-vehicledetection3
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        250        454      0.651      0.586      0.588      0.441



30 epochs completed in 0.206 hours.
Optimizer stripped from runs/detect/yolov8-vehicledetection3/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/yolov8-vehicledetection3/weights/best.pt, 6.2MB

Validating runs/detect/yolov8-vehicledetection3/weights/best.pt...
Ultralytics 8.3.23 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:03<00:00,  4.74it/s]


                   all        250        454      0.618      0.608      0.594      0.452
             Ambulance         50         64      0.804      0.906      0.861      0.715
                   Bus         30         46      0.732      0.711      0.671      0.568
                   Car         90        238      0.573      0.479      0.492      0.346
            Motorcycle         42         46      0.555      0.542      0.535       0.34
                 Truck         38         60      0.427        0.4      0.412      0.293
Speed: 0.3ms preprocess, 2.7ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to runs/detect/yolov8-vehicledetection3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c436d2c29b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

In [9]:
results = model.val(data=yaml)

Ultralytics 8.3.23 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /root/.cache/kagglehub/datasets/alkanerturan/vehicledetection/versions/3/VehiclesDetectionDataset/valid/labels.cache... 250 images, 0 backgrounds, 0 corrupt: 100%|██████████| 250/250 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:05<00:00,  6.27it/s]


                   all        250        454      0.617      0.606      0.594      0.453
             Ambulance         50         64      0.804      0.906      0.861      0.717
                   Bus         30         46      0.732      0.711      0.671      0.568
                   Car         90        238      0.569      0.471      0.492      0.347
            Motorcycle         42         46      0.555      0.543      0.536      0.342
                 Truck         38         60      0.427        0.4      0.413      0.294
Speed: 0.4ms preprocess, 6.5ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to runs/detect/yolov8-vehicledetection32


In [15]:
def test_images(test_directory, model):
    images = os.listdir(test_directory)
    for image_name in images:
        # Filtrando apenas arquivos de imagem
        if not (image_name.endswith('.jpg') or image_name.endswith('.png')):
            continue

        image_path = os.path.join(test_directory, image_name)
        img = cv2.imread(image_path)

        # Verifica se a imagem foi carregada corretamente
        if img is None:
            print(f"Erro ao carregar a imagem: {image_path}")
            continue

        # Realizando a predição
        results = model.predict(source=img)

        # Exibindo a imagem original e as detecções
        for result in results:
            boxes = result.boxes.xyxy
            for box in boxes:
                x1, y1, x2, y2 = map(int, box[:4])
                class_index = int(box[5]) if len(box) > 5 else -1
                class_name = result.names[class_index] if class_index >= 0 else 'Unknown'

                # Desenhando a caixa delimitadora
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(img, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        # Exibindo a imagem com as detecções
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()

# Testando as imagens do conjunto de teste
test_images(test, model)

In [10]:
import cv2

def analyze_video(input_video, trained_model):
    video_capture = cv2.VideoCapture(input_video)
    output_path = '/content/Video.mp4'
    video_writer = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 20.0,
                                   (int(video_capture.get(3)), int(video_capture.get(4))))

    while video_capture.isOpened():
        success, frame = video_capture.read()
        if not success:
            break

        detection_results = trained_model.predict(source=frame)
        for detection in detection_results:
            bounding_boxes = detection.boxes.xyxy
            for box in bounding_boxes:
                x_min, y_min, x_max, y_max = map(int, box[:4])
                if len(box) > 5:  # Verifica se há informações de classe
                    class_id = int(box[5])  # Assume que o índice da classe está em box[5]
                    class_label = detection.names[class_id]
                    cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)
                    cv2.putText(frame, class_label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX,
                                0.9, (255, 0, 0), 2)
                else:
                    # Caso em que as informações da classe podem não estar disponíveis
                    cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)

        video_writer.write(frame)

    video_capture.release()
    video_writer.release()
    print(f"Video processado e salvo em: {output_path}")

analyze_video(video, model)


A saída de streaming foi truncada nas últimas 5000 linhas.
Speed: 2.1ms preprocess, 11.8ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 Cars, 1 Motorcycle, 10.7ms
Speed: 6.1ms preprocess, 10.7ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 Cars, 1 Motorcycle, 13.8ms
Speed: 3.7ms preprocess, 13.8ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 Cars, 1 Motorcycle, 13.0ms
Speed: 2.1ms preprocess, 13.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Cars, 1 Motorcycle, 15.3ms
Speed: 2.0ms preprocess, 15.3ms inference, 8.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Cars, 1 Motorcycle, 9.9ms
Speed: 2.2ms preprocess, 9.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 Cars, 1 Motorcycle, 19.8ms
Speed: 2.0ms preprocess, 19.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 Cars

In [16]:
# Salvar o modelo treinado no diretório padrão do Colab
model_path = "/content/yolov8-vehicledetection.pt"  # Caminho onde o modelo será salvo
model.save(model_path)  # Salva o modelo

print(f"Modelo salvo em: {model_path}")

Modelo salvo em: /content/yolov8-vehicledetection.pt
